On cherche à étudier la probabilité de survie des passagers du Titanic en fonction des informations que l'on a trouvé sur eux.
Dans un premier temps, on peut regarder quels sont les types d'informations que l'on a, est ce que les données sont de bonne qualité et les améliorer si nécessaire.
Dans un second temps, on peut alors extraire des descripteurs de ces données.
Enfin, on peut construire un modèle de régression qui se base sur les descripteurs et ainsi prédire les probabilités de survie.

On dispose de deux fichiers : train.csv et test.csv. On peut regrouper les deux fichiers en un seul pour étudier les données car ainsi on pourra mieux nettoyer le fichier test en prenant en compte les informations du fichier train :


In [69]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
train_ = pd.read_csv('train.csv')
test_= pd.read_csv('test.csv')
test_['Survived'] = np.nan

data = pd.concat([train_,test_], axis=0)
data.head(3)
data.count()

Age            1046
Cabin           295
Embarked       1307
Fare           1308
Name           1309
Parch          1309
PassengerId    1309
Pclass         1309
Sex            1309
SibSp          1309
Survived        891
Ticket         1309
dtype: int64

On peut auditer la qualité des données en 5 temps:
-validité : est ce que les données sont bien formattées ?
-exactitude : est ce que les données ont les bonnes valeurs ?
-consistence : est ce que les données sont cohérentes entre elles ?
-uniformité : est ce que les mêmes unités sont utilisées ?
-complétude : est ce que les données sont complètes ? 


In [70]:
#Validity
print('Unique elements of the Sex column:')
print(data['Sex'].unique())
print('Unique elements of the Embarked column:')
print(data['Embarked'].unique())
print('Unique elements of the SibSp column:')
print(data['SibSp'].unique())
print('Unique elements of the Parch column:')
print(data['Parch'].unique())
print('Unique elements of the Pclass column:')
print(data['Pclass'].unique())
print('Unique elements of the Cabin column:')
print(data['Cabin'].unique())
print('Unique elements of the Fare column:')
print(data['Fare'].unique())
print(len(data['PassengerId'].unique()))
print('Unique elements of the Age column:')
print(data['Age'].unique())
print('Unique elements of the Survived column:')
print(data['Survived'].unique())
pd.options.display.max_colwidth = 100
print('10 values of the Name column:')
print(data['Name'][20:30])
print('First 10 values of the Ticket column:')
print(data['Ticket'][:10])

Unique elements of the Sex column:
['male' 'female']
Unique elements of the Embarked column:
['S' 'C' 'Q' nan]
Unique elements of the SibSp column:
[1 0 3 4 2 5 8]
Unique elements of the Parch column:
[0 1 2 5 3 4 6 9]
Unique elements of the Pclass column:
[3 1 2]
Unique elements of the Cabin column:
[nan 'C85' 'C123' 'E46' 'G6' 'C103' 'D56' 'A6' 'C23 C25 C27' 'B78' 'D33'
 'B30' 'C52' 'B28' 'C83' 'F33' 'F G73' 'E31' 'A5' 'D10 D12' 'D26' 'C110'
 'B58 B60' 'E101' 'F E69' 'D47' 'B86' 'F2' 'C2' 'E33' 'B19' 'A7' 'C49' 'F4'
 'A32' 'B4' 'B80' 'A31' 'D36' 'D15' 'C93' 'C78' 'D35' 'C87' 'B77' 'E67'
 'B94' 'C125' 'C99' 'C118' 'D7' 'A19' 'B49' 'D' 'C22 C26' 'C106' 'C65'
 'E36' 'C54' 'B57 B59 B63 B66' 'C7' 'E34' 'C32' 'B18' 'C124' 'C91' 'E40'
 'T' 'C128' 'D37' 'B35' 'E50' 'C82' 'B96 B98' 'E10' 'E44' 'A34' 'C104'
 'C111' 'C92' 'E38' 'D21' 'E12' 'E63' 'A14' 'B37' 'C30' 'D20' 'B79' 'E25'
 'D46' 'B73' 'C95' 'B38' 'B39' 'B22' 'C86' 'C70' 'A16' 'C101' 'C68' 'A10'
 'E68' 'B41' 'A20' 'D19' 'D50' 'D9' 'A23'

On voit avec la fonction unique() qui retourne toutes les valeurs différentes d'une variable, que la colonne Sex a de bonnes valeurs,on a soit male soit female. 
La colonne Embarked est aussi valide (à part les valeurs manquantes on a soit C soit S soit Q).
Les colonnes SibSp, Parch, Pclass, Fare et Age, Survived contiennent toutes des entiers ou des réels (sans tenir compte des données manquantes).
Pour les valeurs présentes dans la colonne Cabin, on voit par contre que certaines sont de la forme lettre chiffre (C85) alors que pour d'autres correspondent à une liste de combinaisons lettre chiffre ou simplement lettre.
Compte tenue du faible nombre de valeurs non nulles pour cette colonne, on n'en tient pas compte pour le moment.
La colonne PassengerId fournit bien un identifiant unique au passager.
Certaines valeurs de la colonne Ticket sont des chiffres alors que d'autres ont des lettres devant ce chiffre. Comme le sens des lettres n'est pas évident, on peut tout formater en ne gardant que le chiffre.
Les noms semblent suivre le format : "nom-de-famille,prénom(s)" ou "nom-de-famille,prénom(s)(prénom-alternatif nom alternatif)" si la personne s'est marié et a donc changé son nom de jeune fille ou si on a retrouvé un homme sous deux identités différentes. On a cependant quelques exceptions dans ce format avec des guillements et parfois des surnoms entre guillemets ("nellie" pour ellen ou "annie" pour anna).

On fait les changements de la manière suivante :

In [71]:
data['TicketNumber'] = data['Ticket'].apply(lambda x : x.split(' ')[-1])
def removeQuotes(string):
    if '"' in string:
        if len(string.split('"')[1]) >1:
            if '(' in string or ')' in string or len(string.split('"')[1].split(' '))>1:
                return string.replace('\"','')
            else:
                return ''.join([string.split('"')[0],string.split('"')[2]])
            
        else:
            return string.replace("\"","")
    else:
        return string
data['Name']=data['Name'].apply(lambda x: removeQuotes(x.replace('/','-')))
print('10 values of the Name column:')
print(data['Name'][20:30])
print('First 10 values of the Ticket column:')
print(data['TicketNumber'][:10])

10 values of the Name column:
20                                         Fynney, Mr. Joseph J
21                                        Beesley, Mr. Lawrence
22                                         McGowan, Miss. Anna 
23                                 Sloper, Mr. William Thompson
24                                Palsson, Miss. Torborg Danira
25    Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)
26                                      Emir, Mr. Farred Chehab
27                               Fortune, Mr. Charles Alexander
28                                        O'Dwyer, Miss. Ellen 
29                                          Todoroff, Mr. Lalio
Name: Name, dtype: object
First 10 values of the Ticket column:
0      21171
1      17599
2    3101282
3     113803
4     373450
5     330877
6      17463
7     349909
8     347742
9     237736
Name: TicketNumber, dtype: object


Après cela, on obtient un tableau de données où toutes les valeurs non manquantes d'une colonne ont la même structure.
Comme sur ces données-ci on ne peut pas vérifier l'exactitude des valeurs face à des données de référence, on passe directement à la prochaine étape qui consiste à voir si les données sont cohérentes entre elle, c'est à dire de voir s'il y a des valeurs dans des colonnes différentes qui se contredisent.
On peut par exemple regarder si le sexe de la personne est cohérent avec le titre qui lui est donné.
S'il y a une femme dont le titre indique qu'elle n'est pas marié mais qu'elle a changé de nom (qu'elle a un nom entre parenthèse).
Si les prix sont cohérents en moyenne avec les classes.

In [72]:
def verifySex(df):
    if (df['Name'].split(",")[1].split(".")[0].strip() in ['Don','Sir','Jonkheer','Rev','Major','Col','Capt','Mr','Master']) and (df['Sex']=='male'):
        return True
    elif (df['Name'].split(",")[1].split(".")[0].strip() in ['Dona','Lady','the Countess','Mrs','Miss','Mme','Ms','Mlle']) and (df['Sex']=='female'):
        return True
    else:
        return False
data['ConsistentSex'] = data.apply(lambda df: int(verifySex(df)),axis=1)
data['Surname'] = data['Name'].apply(lambda x: x.split(",")[0].strip())
print(data['Surname'].value_counts()[:10])
SurnamesCount = data['Surname'].value_counts()
data['SurnameCount']=data['Surname'].apply(lambda x:SurnamesCount[x])

def getTitle(string):
    title = string.split(",")[1].split(".")[0]
    if title in ['Don','Lady','Sir','the Countess','Jonkheer','Dona']:
        return 'Noble'
    elif title in ['Major','Col','Capt']:
        return 'Military'
    elif title in ['Miss','Mlle','Ms']:
        return 'Miss'
    elif title in ['Mme','Mrs']:
        return 'Mrs'
    else:
        return title
data['Title'] = data['Name'].apply(lambda x: getTitle(x))
def changedName(df):
    if  ('(' in df['Name']):
        return True
    else:
        return False
data['ChangedName']= data.apply(lambda x: int(changedName(x)),axis=1)
print(data[['Pclass','Name','ChangedName','Title']].loc[(data['ChangedName']==1) & (data['Title']=='Miss')])
print(data['Fare'].loc[data['Pclass']==1].mean())
print(data['Fare'].loc[data['Pclass']==2].mean())
print(data['Fare'].loc[data['Pclass']==3].mean())
print(data['Fare'].loc[data['Pclass']==1].median())
print(data['Fare'].loc[data['Pclass']==2].median())
print(data['Fare'].loc[data['Pclass']==3].median())
#print(data[['Surname','NbMembers','SurnameCount']].loc[data['NbMembers']>data['SurnameCount']])
#print(data.loc[(data['Surname']=='Richards') | (data['Surname']=='Hocking')])

Andersson    11
Sage         11
Goodwin       8
Asplund       8
Davies        7
Ford          6
Brown         6
Skoog         6
Smith         6
Carter        6
Name: Surname, dtype: int64
Empty DataFrame
Columns: [Pclass, Name, ChangedName, Title]
Index: []
87.5089916408668
21.1791963898917
13.302888700564969
60.0
15.0458
8.05


Globalement les données ne semblent pas présenter de contradictions. On peut donc s'interesser à l'uniformité des unités des valeurs et en regardant les résultats précédents quand on applique la fonction unique() aux colonnes, cela semble être le cas, il n'y a pas de valeur aberrante, qui indiquerait une mauvaise unité.
On peut maintenant compléter les données manquantes :

In [73]:
data.count()
data["Embarked"] = data["Embarked"].astype('category')
data['Embarked']=data.groupby(['Sex','Pclass','SibSp'])['Embarked'].transform(lambda x: x.fillna(x.mode()[0]))
data["Fare"] = data.groupby(['Sex','Pclass','Embarked','SibSp'])['Fare'].transform(lambda x: x.fillna(x.mean()))
data['Age'] = data.groupby(['Sex','Pclass','Embarked'])['Age'].transform(lambda x: x.fillna(x.mean()))

print(data.count())

Age              1309
Cabin             295
Embarked         1309
Fare             1309
Name             1309
Parch            1309
PassengerId      1309
Pclass           1309
Sex              1309
SibSp            1309
Survived          891
Ticket           1309
TicketNumber     1309
ConsistentSex    1309
Surname          1309
SurnameCount     1309
Title            1309
ChangedName      1309
dtype: int64


Maintenant que l'on a amélioré autant que possible la qualité des données à notre disposition, on peut analyser les variables pour déterminer quels sont les descripteurs pertinents qui contribueraient à prédire la probabilité de survie.
Les schémas suivants ont été fait avec Tableau :

In [74]:
%%HTML
<div class='tableauPlaceholder' id='viz1508164350110' style='position: relative'><noscript><a href='#'><img alt='Histogram of the count of passenger per sex and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='site_root' value='' /><param name='name' value='titanic_74&#47;Sheet1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508164350110');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1508164375871' style='position: relative'><noscript><a href='#'><img alt='Histogram of the count of passenger per sex, class and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet5&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='site_root' value='' /><param name='name' value='titanic_74&#47;Sheet5' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet5&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508164375871');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1508164420828' style='position: relative'><noscript><a href='#'><img alt='Histogram of the count of passenger per sex, embarking port and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;C7&#47;C7NY687MJ&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='path' value='shared&#47;C7NY687MJ' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;C7&#47;C7NY687MJ&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508164420828');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1508164603995' style='position: relative'><noscript><a href='#'><img alt='Histogram of the count of passenger per age bin, sex and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet11&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='site_root' value='' /><param name='name' value='titanic_74&#47;Sheet11' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet11&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508164603995');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1508164984342' style='position: relative'><noscript><a href='#'><img alt='Histogram of the count of passenger per family size and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet13&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='site_root' value='' /><param name='name' value='titanic_74&#47;Sheet13' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;ti&#47;titanic_74&#47;Sheet13&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508164984342');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>
<div class='tableauPlaceholder' id='viz1508166587856' style='position: relative'><noscript><a href='#'><img alt='Histogram of count of passenger by fare and survived metric ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;TJ&#47;TJRPYHSY7&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='2' /> <param name='path' value='shared&#47;TJRPYHSY7' /> <param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;TJ&#47;TJRPYHSY7&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1508166587856');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Comme la plupart des variables sont des variables catégoriques, il semble plus interessant de faire des histogrammes que des matrices de corrélation.
Le premier schéma nous montre qu'il y a plus d'hommes que de femmes et que la proportion de femmes qui ont survécu est beaucoup plus grande.

Le deuxième schéma montre qu'il y avait plus de passagers en 3ème classe (surtout plus d'hommes) et qu'une proportion plus grande de ceux-ci n'a pas survécu.

Le troisième schéma montre que la plupart des passagers ont embarqué à Southampton avec une chance de survie moindre.

Sur le quatrième schéma on peut voir que les femmes ont une chance de survivre à peut près égale quelque soit leur age alors que pour les hommes, les enfants de moins de 18 ans ont plus de chance de survivre que les autres.

Sur le cinquième schéma, on peut voir que les personnes qui ont entre 1 et 3 membres dans leur famille ont plus de chance de survivre que ceux qui n'en ont pas ou beaucoup plus.

Et enfin sur le dernier schéma on peut voir que plus le prix du ticket est cher plus les personnes ont de chance de survivre.

A partir de ces vues même partielles, on observe que le sexe, la classe, le point d'embarquement, l'age, la taille de la famille et le prix du ticket ont une importance pour la probabilité de survie.

La variable sex peut être simplement binarisée en 0 ou 1.
La variable classe peut être laissée telle quelle en effet même si cette catégorie est représentée par un entier, les distances entre classes socio-économiques sont respectées (1 et plus loin de 3 que 2 ne l'est).
La variable qui décrit le port d'embarquement devra être encodé sur trois dimensions(C en 0,0,1 ; S en 0,1,0 et Q en 1,0,0 par exemple pour avoir des catégories equidistantes).
On peut définir une variable taille de famille FamilleSize = SibSp + Parch.
On peut calculer un prix unitaire de ticket comme le prix affiché divisé par le nombre de personnes voyageant avec le même numéro.

In [75]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
data['SexBinary']= lb.fit_transform(data['Sex'])

data['FamilySize']=data['SibSp']+data['Parch']+1.

dummies_Embarked = pd.get_dummies(data['Embarked'])

countTicket = data['TicketNumber'].value_counts()
data['NbReservations']=data['TicketNumber'].apply(lambda x:countTicket[x])
data['FareByPassenger']=data['Fare']/data['NbReservations']


On a un certain nombre de descripteur, on pourrait chercher à diminuer ce nombre (PCA par exemple) mais comme beaucoup de variable sont des catégories, cela n'est pas adapté.
On va donc directement utiliser ces variables comme descripteurs pour entrainer un model.
Le modèle retenu est un modèle de foret aléatoire, en effet c'est une methode d'ensemble qui globalement marche bien et le modèle est plus facile à paramétrer que d'autres modèles d'ensemble. Il y a deux paramètres pour ce modèle, le nombre d'arbres à créer et le nombre de descripteurs considérés à chaque noeud. Il a l'avantage aussi de ne pas nécessité de normaliser les données.
On divise nos données en deux groupes X_ et X_predict (données en fait issues du fichier test.csv pour lesquelles on veut prédire la probabiité de survie des passagers) puis avec les données X_, on les redivise pour pouvoir tester notre modèle :

In [76]:
data_feature = data[['Age','FareByPassenger','Pclass','FamilySize','Survived','SexBinary']]
data_feature = pd.concat([data_feature,dummies_Embarked],axis=1)
X = data_feature.loc[data_feature['Survived'].notnull()].drop(['Survived'],axis=1)
y = data_feature['Survived'].loc[data_feature['Survived'].notnull()]

data_feature_predict = data[['Age','FareByPassenger','Pclass','FamilySize','Survived','SexBinary','PassengerId']]
data_feature_predict = pd.concat([data_feature_predict,dummies_Embarked],axis=1)
X_predict = data_feature_predict.loc[data_feature_predict['Survived'].isnull()].drop(['Survived'],axis=1)

X_, X_test, y_, y_test = train_test_split(X, y, test_size=.3, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=0)

print(X_train.shape)


(623, 8)


On peut alors entrainer un modèle sur les données X_train. On utiliser la fonction GridSearchCV pour faire une recherche des meilleurs paramètres du modèle à utiliser, cette fonction utilise la cross-validation c'est à dire que pour chaque configuration, on divise les données en n groupes ; pour chaque groupe on entraine le modèle sur les n-1 autres groupes et on test le modèle sur le groupe restant et en faisant cela pour chaque groupe on peut avoir une moyenne de performance de régression.
On peut ensuite regarder les performances du meilleur modèle sur les données X_test. Comme on cherche des probabilités à partir de labels binaires, il est necessaire de seuiller pour pouvoir comparer avec les labels des données test.

In [77]:
forest_params = {'n_estimators':range(10,201,10),'max_features':['auto','sqrt','log2']}
rfc = GridSearchCV(RandomForestRegressor(max_features='auto'), forest_params, cv=5)
rfc.fit(X_train, y_train)

print(rfc.best_score_)                                 

for param_name in sorted(forest_params.keys()):
    print("%s: %r" % (param_name, rfc.best_params_[param_name]))

rf_scores = cross_val_predict(rfc.best_estimator_, X_train, y_train, cv=5)
#print(rf_scores)
print(rf_scores.mean())
print('Importance des variables:')
print(rfc.best_estimator_.feature_importances_)

predicted_RF = rfc.predict(X_test)
print("RF part, metrics on test set:")
print(metrics.classification_report(y_test, pd.Series(predicted_RF>0.5).apply(lambda x: int(x))))


0.392034047912
max_features: 'log2'
n_estimators: 160
0.384870470537
Importance des variables:
[ 0.28270608  0.25399216  0.06781401  0.08230862  0.27487142  0.01323067
  0.00820871  0.01686833]
RF part, metrics on test set:
             precision    recall  f1-score   support

        0.0       0.82      0.86      0.84       168
        1.0       0.75      0.69      0.72       100

avg / total       0.80      0.80      0.80       268



Les variables les plus importantes sont le sexe, l'age et le prix du ticket ce qui confirme les histogrammes montrés précédemment.
On peut alors vérifier que le modèle marche dans un cas général en le testant sur les données X_valid qui n'ont jamais été utilisées pour paramétrer le modèle.
Si le résultat est satisfaisant on peut prédire la probabilité pour les données sans probabilité de survie, probabilités qu'on enregistre dans le fichier prediction.csv.

In [78]:
valid_RF = rfc.predict(X_valid)
print("RF part, metrics on valid set:")
print(metrics.classification_report(y_valid, pd.Series(valid_RF>0.5).apply(lambda x: int(x))))

X_predict['Survived'] = rfc.predict(X_predict.drop(['PassengerId'],axis=1))
X_predict[['PassengerId','Survived']].to_csv('prediction-marc-duda.csv',index=False)

RF part, metrics on valid set:
             precision    recall  f1-score   support

        0.0       0.82      0.86      0.84       168
        1.0       0.75      0.69      0.72       100

avg / total       0.80      0.80      0.80       268

